In [2]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Lambda, Conv2D, Dropout, Dense, Flatten
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

from PIL import Image
import os

Using TensorFlow backend.


In [3]:
path = './data/'
imgs_path = path + 'imgs/'

file = path + 'data.csv'

df = pd.read_csv(file, names=['img', 'left', 'nothing', 'right'])

#df['steering'][1:] = df['steering'][1:].apply(lambda x: round(float(x)))
df['left'][1:] = df['left'][1:].apply(lambda x: int(x))
df['nothing'][1:] = df['nothing'][1:].apply(lambda x: int(x))
df['right'][1:] = df['right'][1:].apply(lambda x: int(x))

In [15]:
left = []
nothing = []
right = []

for data in df.values[1:]:
    img = data[0]
    action = data[1:]
    
    if action[0] == 1:
        left.append([img, [1, 0, 0]])
    elif action[1] == 1:
        nothing.append([img, [0, 1, 0]]) 
    elif action[2] == 1:
        right.append([img, [0, 0, 1]])
    else:
        print("jio")

counter = [len(left), len(nothing), len(right)]

In [17]:
limit = min(counter)

In [18]:
left = left[:limit]
nothing = nothing[:limit]
right = right[:limit]

In [19]:
training_data = left + nothing + right
training_data = np.array(shuffle(training_data))

In [21]:
training_df = pd.DataFrame(training_data)

In [23]:
X, y = training_df[0], training_df[1]

In [24]:
X_imgs = []
for img in X.values:
    im = Image.open(img)
    X_imgs.append(np.array(im))
    im.close()

X = np.array(X_imgs)

In [25]:
y_ = np.array([np.array(i) for i in y])

#y = to_categorical(y_)

In [27]:
model = Sequential()
model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(300, 800, 3)))
model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_json = model.to_json()
with open("./data/models/model.json", "w") as json_file:
    json_file.write(model_json)

checkpoint = ModelCheckpoint('./data/models/model-{epoch:03d}-{accuracy:03f}-{val_accuracy:03f}.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='auto')  

model.fit(X, y_, batch_size=6, epochs=15, validation_split=0.2, callbacks=[checkpoint], verbose=1)

c:\users\javie\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2))`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\javie\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
  after removing the cwd from sys.path.
c:\users\javie\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
  """
c:\users\javie\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  
c:\users\javie\appdata\local\programs\pyt

Train on 105 samples, validate on 27 samples
Epoch 1/15
105/105 [==============================] - 14s 132ms/step - loss: 4.1880 - accuracy: 0.4095 - val_loss: 4.6559 - val_accuracy: 0.5185

Epoch 00001: val_loss improved from inf to 4.65588, saving model to ./data/models/model-001-0.409524-0.518519.h5
Epoch 2/15
105/105 [==============================] - 15s 140ms/step - loss: 3.7252 - accuracy: 0.5524 - val_loss: 2.6164 - val_accuracy: 0.4074

Epoch 00002: val_loss improved from 4.65588 to 2.61637, saving model to ./data/models/model-002-0.552381-0.407407.h5
Epoch 3/15
105/105 [==============================] - 13s 128ms/step - loss: 1.2716 - accuracy: 0.6095 - val_loss: 2.1074 - val_accuracy: 0.4074

Epoch 00003: val_loss improved from 2.61637 to 2.10740, saving model to ./data/models/model-003-0.609524-0.407407.h5
Epoch 4/15
105/105 [==============================] - 13s 124ms/step - loss: 2.1423 - accuracy: 0.6381 - val_loss: 1.2048 - val_accuracy: 0.2963

Epoch 00004: val_loss im